In [527]:
import plotly.graph_objects as go
import numpy as np
import torch
import xarray

In [528]:
gmb = xarray.open_dataset(
    "GRAVIS-3_2002095-2023334_COSTG_0100_AIS_GRID_TUD_0003.nc")

In [529]:
# Visualise any time slice
fig = go.Figure(data = go.Heatmap(
                   z = gmb.dm[1],
                   x = gmb.dm.x,
                   y = gmb.dm.y,
                   colorscale = "IceFire"))

fig.update_layout(
    autosize = False,
    width = 1100,
    height = 1000)

fig.update_layout(template = "simple_white")

fig.show()

In [530]:
# extract corner points
x = np.array([-2.5, -2.25, -1.55, -1.3, -0.65, -0.5, -0.75, -0.3, 0.9, 1.35, 2.2, 2.25, 1.75, 2.15, 2.6, 2.65, 2.0, 1.5, 0.9, 0.35, 0.4, -0.1, -0.55, -0.5, -1.2, -1.6, -1.55, -1.8, -1.75, -2.35, -2.5]) * 1e6
y = np.array([1.65, 1.1, 0.9, 0.3, 0.4, 0.95, 1.1, 2.1, 1.85, 1.95, 1.55, 0.85, 0.7, 0.65, 0.1, -0.65, -1.65, -2.0, -2.1, -1.95, -1.05, -0.55, -0.9,-1.3, -1.15, -0.6, -0.25, -0.4, 0.8, 1.05, 1.65]) * 1e6

In [531]:
# Visualise any time slice
fig = go.Figure(data = go.Scatter(
    x = x,
    y = y, 
    # fill = "toself",
    line = dict(color = '#0061ff',
                width = 8
                )))

fig.update_layout(
    autosize = False,
    width = 1100,
    height = 1000)

# margin
fig.update_yaxes(range = [y.min() - 1e5, y.max() + 1e5])
fig.update_xaxes(range = [x.min() - 1e5, x.max() + 1e5])

# remove grid and axis
fig.update_layout(template = "simple_white")
fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.show()

# Splines

In [498]:
# Visualise any time slice
fig = go.Figure(data = go.Scatter(
    x = x,
    y = y,
    mode = "markers+lines",
    marker = dict(
        color = 'forestgreen',
        size = 10),
    line = dict(
        color = 'lightgreen',
        width = 2,
        shape = "spline",
        smoothing = 1.3),
    ))

fig.add_trace(
    go.Scatter(
    x = x,
    y = y,
    mode = "markers+lines",
    marker = dict(
        color = 'forestgreen',
        size = 10),
    line = dict(
        color = 'lightgreen',
        width = 2,
        shape = "spline",
        smoothing = 0.3),
    )
)

fig.add_trace(
    go.Scatter(
    x = x,
    y = y,
    mode = "markers+lines",
    marker = dict(
        color = 'forestgreen',
        size = 10),
    line = dict(
        color = 'lightgreen',
        width = 2,
        shape = "spline",
        smoothing = 0.0),
    )
)

fig.update_layout(
    autosize = False,
    width = 600,
    height = 600)

# margin
fig.update_yaxes(range = [y.min() - 1e5, y.max() + 1e5])
fig.update_xaxes(range = [x.min() - 1e5, x.max() + 1e5])

# remove grid and axis
fig.update_layout(template = "simple_white")
fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.show()

# GP

- tranform points onto 1D line where 
- GP sample the deviations from the line

data points are given as such  
query points are lines?!

Transform to points along 1 D:
(a string we are pulling)

In [532]:
xt = torch.tensor(x)
yt = torch.tensor(y)
# xyt.shape: torch.Size([2, 31])
xyt = torch.tensor(torch.cat((xt.unsqueeze(0), yt.unsqueeze(0))))

/tmp/ipykernel_1503283/769244679.py:4: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [533]:
# pairwise distance
distances = torch.cdist(xyt.mT, xyt.mT)

# we only consider the along track distance
linear_distances = torch.empty(size = (1, 0))

for i in range(distances.shape[0] - 1):
    # print(distances[i, i + 1])
    linear_distances = torch.cat((linear_distances, distances[i, i + 1].unsqueeze(0).unsqueeze(0)), dim = 1)

In [534]:
linear_x = torch.cumsum(linear_distances, dim = 1)
# add zero for first dot
linear_x = torch.cat((torch.tensor([0.0]).unsqueeze(0), linear_x), dim = 1)
# define all as 0: mean would be a straight line
linear_y = torch.tensor([0.0] * linear_x.shape[1])

In [535]:
fig = go.Figure(data = go.Scatter(
    x = np.array(linear_x.squeeze()),
    y = np.array(linear_y.squeeze()),
    mode = "markers+lines"))

fig.update_layout(template = "simple_white")

fig.show()

## GP

https://krasserm.github.io/2018/03/19/gaussian-processes/

In [537]:
from numpy.linalg import inv

def kernel(X1, X2, l = 1.0, sigma_f = 1.0):
    """
    Isotropic squared exponential kernel.
    
    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        (m x n) matrix.
    """
    # X1**2 
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def posterior(X_s, X_train, Y_train, l = 1.0, sigma_f = 1.0, sigma_y = 1e-8):
    """
    Computes the suffifient statistics of the posterior distribution 
    from m training data X_train and Y_train and n new inputs X_s.
    
    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_y: Noise parameter.
    
    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    """
    K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
    K_inv = inv(K)
    
    # Equation (7)
    mu_s = K_s.T.dot(K_inv).dot(Y_train)

    # Equation (8)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
    
    return mu_s, cov_s

In [538]:
np.array(linear_x.squeeze())
np.array(linear_y.squeeze())
step_size = 100_000
x_query = np.arange(0, linear_x[0, -1].item(), step_size)
x_query.shape

(204,)

In [540]:
mu_s, cov_s = posterior(
    x_query.reshape(-1, 1), # shape N, 1
    np.array(linear_x.squeeze()).reshape(-1, 1),
    np.array(linear_y.squeeze()).reshape(-1, 1),
    l = 400_000, # l = 500_000,
    sigma_y = 0.00001, # sigma_y = 0.09
    sigma_f = 100) # # sigma_f = 1.0

n_samples = 12
samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, n_samples)

fig = go.Figure(data = go.Scatter(
    x = np.array(linear_x.squeeze()),
    y = np.array(linear_y.squeeze()),
    mode = "markers+lines"))

for i in range(samples.shape[0]):
    fig.add_trace(
        go.Scatter(
    x = x_query,
    y = samples[i])
    )

fig.update_layout(template = "simple_white")

fig.show()

## Given the flat samples, translate back

In [541]:
samples_t = torch.cat((torch.tensor(x_query).unsqueeze(0), torch.tensor(samples)))

In [548]:
sampled_outlines = torch.empty(size = (n_samples, 2, 0))
make_orth = torch.tensor([-1, 1])

for i in range(linear_x.shape[1] - 1):
    # i iterated through all starting points

    # torch.Size([1, 31])
    start_1d = linear_x[0, i]
    end_1d = linear_x[0, i + 1]

    # torch.Size([2, 31])
    start_2d = xyt[:, i]
    end_2d = xyt[:, i + 1]

    # subset samples that are within the segment, remove x_query column
    samples_segment = samples_t[:,(samples_t[0] >= start_1d) & (samples_t[0] <= end_1d)][1:,:]

    # offset in x and y direction needed to get from start to finish
    step_vector = end_2d.sub(start_2d)

    # Normalise lenth of step to step size
    # pythagoras
    unit_step_vector = step_vector.div(step_vector.square().sum().sqrt().div(step_size))
    orth_unit_step_vector = unit_step_vector.mul(make_orth)

    # Add start_2d
    sampled_outlines = torch.cat((sampled_outlines, start_2d.unsqueeze(0).tile(n_samples, 1).unsqueeze(-1)), dim = -1)

    # take unit steps
    for j in range(samples_segment.shape[-1]):
        # Add 1 for multiplication
        regular_step = unit_step_vector.mul(j).tile(n_samples, 1)

        offset_step = samples_segment[:, j].mul(0.02).unsqueeze(-1).mul(orth_unit_step_vector)

        step_samples = regular_step + offset_step

        # Create copies for n_sampled and keep 2 xy dims
        # unsqueeze for concat
        position_samples = start_2d.tile((n_samples, 1)).add(step_samples).unsqueeze(-1)

        sampled_outlines = torch.cat((sampled_outlines, position_samples), dim = -1)

    # Add end_2d
    sampled_outlines = torch.cat((sampled_outlines, end_2d.unsqueeze(0).tile(n_samples, 1).unsqueeze(-1)), dim = -1)

In [478]:
selected_colors = ["#5084F6", "#2CB005", "#E50959", "#8B09E5", "#E57009", "#E57009", "#5084F6", "#2CB005", "#E50959", "#8B09E5", "#E57009", "#E57009"]

In [479]:
# Visualise any time slice
fig = go.Figure(data = go.Scatter(
    x = x,
    y = y,
    mode = "lines",
    marker = dict(
        color = 'forestgreen',
        size = 10),
    ))

for s in range(sampled_outlines.shape[0]):
    fig.add_trace(
        go.Scatter(
        x = sampled_outlines[s, 0, :],
        y = sampled_outlines[s, 1, :],
        mode = "lines",
        line = dict(
            color = selected_colors[s],
            width = 2,
            shape = "spline",
            smoothing = 0.0),
        )
    )

fig.update_layout(
    autosize = False,
    width = 1200,
    height = 1000)

# margin
# fig.update_yaxes(range = [y.min() - 1e5, y.max() + 1e5])
# fig.update_xaxes(range = [x.min() - 1e5, x.max() + 1e5])

# remove grid and axis
fig.update_layout(template = "simple_white")
fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.show()

In [512]:
selected_greens = [
    "#42FF00", 
    "#1ECF1E", 
    "#BFF70D", 
    "#DEF70D", 
    "#D1DC25", 
    "#4CBB03", 
    "#53F94E", 
    "#63F573", 
    "#51D639", 
    "#86F03D", 
    "#5A9E2B", 
    "#53B60F"]

In [549]:
# Visualise any time slice
fig = go.Figure(data = go.Scatter(
    x = x,
    y = y,
    mode = "lines",
    marker = dict(
        color = 'forestgreen',
        size = 10),
    ))

for s in range(sampled_outlines.shape[0]):
    fig.add_trace(
        go.Scatter(
        x = sampled_outlines[s, 0, :],
        y = sampled_outlines[s, 1, :],
        mode = "lines",
        line = dict(
            color = selected_greens[s],
            width = 2,
            shape = "spline",
            smoothing = 0.0),
        )
    )

fig.add_trace(
        go.Scatter(
        x = x,
        y = y,
        mode = "markers",
        marker = dict(
            color = "#63FE0A",
            size = 15),
        )
    )

fig.update_layout(
    autosize = False,
    width = 700,
    height = 650)

# margin
# fig.update_yaxes(range = [y.min() - 1e5, y.max() + 1e5])
# fig.update_xaxes(range = [x.min() - 1e5, x.max() + 1e5])

# remove grid and axis
fig.update_layout(template = "simple_white")
fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)
fig.update_layout(showlegend = False)

fig.update_layout(
    # paper_bgcolor = 'rgba(0,0,0,0)',
    # plot_bgcolor = 'rgba(0,0,0,0)',
    paper_bgcolor = '#144215',
    plot_bgcolor = '#144215',
)

fig.show()

In [314]:
# Visualise any time slice
fig = go.Figure(data = go.Scatter(
    x = x,
    y = y,
    mode = "markers+lines",
    marker = dict(
        color = 'forestgreen',
        size = 10),
    line = dict(
        color = 'forestgreen',
        width = 2,
        shape = "spline",
        smoothing = 0.0),
    ))

for s in range(sampled_outlines.shape[0]):
    fig.add_trace(
        go.Scatter(
        x = sampled_outlines[s, 0, :],
        y = sampled_outlines[s, 1, :],
        mode = "lines",
        line = dict(
            color = selected_colors[s],
            width = 2,
            shape = "spline",
            smoothing = 0.0),
        )
    )

fig.update_layout(
    autosize = False,
    width = 600,
    height = 600)

# margin
# fig.update_yaxes(range = [y.min() - 1e5, y.max() + 1e5])
# fig.update_xaxes(range = [x.min() - 1e5, x.max() + 1e5])

# remove grid and axis
fig.update_layout(template = "simple_white")
fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.show()